In [91]:
print('hello world')

hello world


In [99]:

import os
import pandas as pd
from gensim.models import Word2Vec
import optuna
import spacy
from sklearn.cluster import AgglomerativeClustering
from itertools import product
from gensim.models import Word2Vec
import hdbscan
import matplotlib.pyplot as plt
from pathlib import Path

In [95]:
os.chdir(r"C:/BA_thesis/BA_v2_31.03")

print(f"working directory: {os.getcwd()}")
input_path = os.getcwd() + '/files/corpus_data'

corpus = ld.TxtSubdirsCorpus("files/dfs")

working directory: C:\BA_thesis\BA_v2_31.03
Corpus active! Path: files/dfs 



In [100]:
def iter_large_df(top_dir):
    print(f"\nIterating documents in {top_dir}...")
    for root, _, files in os.walk(top_dir):
        for file in files:
            if file.endswith(".pkl"):
                file_path = os.path.join(root, file)
                df = pd.read_pickle(file_path)
                for _, row in df.iterrows():
                    yield row


def merge_df(dir_dfs: str) -> pd.DataFrame:
    path = Path(dir_dfs)
    assert os.path.exists(path), f"Provided path {dir_dfs} does not exist"
    assert os.path.isdir(path), f"Provided path {dir_dfs} is not dir path!"

    gen = iter_large_df(dir_dfs)

    # temporary date container
    d = []
    for r in gen:
        d.append([
            r["clean_text"],
            r["semantic_id"],
            r["date"],
            r["group"]
        ])

    df_full = pd.DataFrame(d, columns=["clean_text", "semantic_id", "date", "group"])
    print("\nDfs merged sucesfully!")
    return df_full


In [102]:
df = merge_df('files/dfs')
df.head


Iterating documents in files/dfs...

Dfs merged sucesfully!


<bound method NDFrame.head of                                              clean_text semantic_id  \
0     [inductive, reasoning, core, component, human,...   254926851   
1     [inductive, reasoning, reach, hypothesis, usua...   254926851   
2     [overcome, challenge, present, novel, paradigm...   254926851   
3     [natural, language, representation, PLMs, reas...   254926851   
4     [plant, carnivorous, probably, trap, structure...   254926851   
...                                                 ...         ...   
1747  [islamic, education, civic, need, consider, as...   254296780   
1748  [Integrative, learning, Islamic, Religious, Ed...   254296780   
1749  [pai, science, learning, use, mix, integrate, ...   254296780   
1750  [crisis, modern, man, light, Masaryk, national...   254294750   
1751  [abstract, beginning, thought, Thomas, Garrigu...   254294750   

            date       group  
0     2022-12-21  psychology  
1     2022-12-21  psychology  
2     2022-12-21  psycho

In [ ]:

    # add vector represenation to each text
    dm.add_document_vector(df, model)

    # extract the dimentions for reduction
    vec = dm.x_from_df(df, 'vector')

    # reduce the dimentions
    df = dm.reduce_dimentionality(df_vectors=vec,
                                  df_normal=df,
                                  rdims=2)